* 자주등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공

In [1]:
!pip install tokenizers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 3.3/3.3 MB 16.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. BERT의 워드피스 토크나이저(BertWordPieceTokenizer)

* 구글의 BERT 모델에 사용한 WordPiece Tokenizer
* 허깅페이스는 해당 토크나이저를 직접 구현하여 tokenizers라는 패키지를 통해 BertWordPieceTokenizer 제공

In [2]:
# 네이버 영화 리뷰 데이터 로드
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x1dbab6c22e0>)

* 센텐스피스(SentencePiece) 실습에서 진행한 전처리 작업과 동일하게 진행

In [5]:
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')

with open("naver_review.txt", 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

* BertWordPiece Tokenizer 설정

In [9]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

* lowercase : 대소문자 구분 여부
* strip_accents : True일 경우 악센트 제거

-----
데이터를 학습하여 단어집합 구함

In [10]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file, vocab_size=vocab_size, 
                limit_alphabet=limit_alphabet, min_frequency=min_frequency)

* limt_alphabet : 병합 전의 초기 토큰의 허용 갯수
* min_frequency : 등장 횟수의 최소값으로 이 경우에만 병합 대상이 됨

----
학습이 완료되면 vocab을 저장함. 경로를 지정해야 함

In [11]:
# vocab 저장
tokenizer.save_model('./')

['./vocab.txt']

In [13]:
# vocab 로드
vocab_df = pd.read_fwf('vocab.txt', header=None)
vocab_df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,말안
29996,말들이
29997,말라는
29998,말밖에는


In [21]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :', encoded.tokens)
print('정수 인코딩 : ', encoded.ids)
print('디코딩 :', tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 :  [2111, 20628, 3673, 3274, 24681, 7873, 7379]
디코딩 : 아 배고픈데 짜장면먹고싶다


In [22]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :', encoded.tokens)
print('정수 인코딩 : ', encoded.ids)
print('디코딩 :', tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 :  [12825, 25645, 3311, 12696, 3319, 10784, 3342]
디코딩 : 커피 한잔의 여유를 즐기다


### 2. 기타 토크나이저

* ByteLevelBPETokenizer : BPE의 바이트 레벨 버전

* CharBPEToknizer : 오리지널 BPE

* SentenceBPETokenizer : SentencePiece와 호환되는 BPE 구현


In [23]:
from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer

In [24]:
tokenizer = SentencePieceBPETokenizer()
tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['▁이', '▁영화는', '▁정말', '▁재미있', '습니다.']
